In [1]:
%matplotlib inline

# Imports

In [2]:
import pandas as pd
import numpy as np

# Load in the clean *E. coli* `.tsv` file


In [3]:
df = pd.read_csv('../Data/NCBI_phage_db/host_genomes/562.clean.tsv', sep='\t')
df.head()

,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,sec_struct_bound,iCUB,GC_cds,GC_upstream,Start_accessibility
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,-12.81,55.949262,0.530654,0.433333,-8.50
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,-14.05,56.062386,0.562701,0.500000,-7.82
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,-20.71,53.052776,0.528361,0.566667,-3.73
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,-7.38,50.705530,0.538721,0.400000,-9.77
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,-8.52,53.158862,0.496782,0.466667,-10.08


**Some later data will require the "Gene" merge so make it a dedicated column and filter any duplicates**

In [4]:
print(df.shape)
df = df.drop_duplicates('gene_id', keep=False)
print(df.shape)
df = df.drop_duplicates('locus_tag', keep=False)
print(df.shape)

(4018, 15)
(4018, 15)
(4018, 15)


# Add in knowledge of protein abundances and join the dataframes

This comes from PaxDB (release 4.1). See:

https://pax-db.org/

and the associated paper:

https://doi.org/10.1002/pmic.201400441

In [5]:
prot_df = pd.read_csv('../Data/562_associated_data/511145-WHOLE_ORGANISM-integrated.txt',\
                      sep='\t', comment='#', header=None)
print(prot_df.shape)

###Add in a dedicated `locus_tag` column by splitting column 1
prot_df['locus_tag']= prot_df[1].str.split(".", n = 1, expand = True)[1]
assert prot_df['locus_tag'].value_counts().max() == 1

###Rename the columns and drop the unnecessary bits
prot_df = prot_df.drop([0, 1], axis=1)
prot_df.columns = ['prot_abundance', 'locus_tag']

###Merge with the original dataframe on `locus_tag`
combined_df = df.merge(prot_df, on='locus_tag', how='left')
print(combined_df.shape)

###Not sure how 0.0 protein abundances are determined / make sense so let's change them to NaN
print(combined_df[combined_df['prot_abundance']==0.0].shape)
combined_df.at[combined_df[combined_df['prot_abundance']==0.0].index, 'prot_abundance'] = np.nan
print(combined_df[combined_df['prot_abundance'].isnull()].shape)
combined_df.head()

(4096, 3)
(4018, 16)
(310, 16)
(440, 16)


,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,sec_struct_bound,iCUB,GC_cds,GC_upstream,Start_accessibility,prot_abundance
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,-12.81,55.949262,0.530654,0.433333,-8.50,317.0
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,-14.05,56.062386,0.562701,0.500000,-7.82,76.3
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,-20.71,53.052776,0.528361,0.566667,-3.73,530.0
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,-7.38,50.705530,0.538721,0.400000,-9.77,NaN
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,-8.52,53.158862,0.496782,0.466667,-10.08,47.9


# Add in translation efficiency

This data comes from two separate data sources:

https://doi.org/10.1016/j.cell.2014.02.033

and more recently:

https://doi.org/10.15252/msb.20188719

In [6]:
teff_df = pd.read_excel('../Data/562_associated_data/Li_et_al_2014.xlsx')
print(teff_df.shape)
teff_df = teff_df.drop(['mRNA level (RPKM)'], axis=1)
teff_df.columns = ['gene_id', 'trans_eff_2014']

combined_df = combined_df.merge(teff_df, on='gene_id', how='left')
print(combined_df.shape)
combined_df.head()

(4095, 3)
(4018, 17)


,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,sec_struct_bound,iCUB,GC_cds,GC_upstream,Start_accessibility,prot_abundance,trans_eff_2014
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,-12.81,55.949262,0.530654,0.433333,-8.50,317.0,0.98
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,-14.05,56.062386,0.562701,0.500000,-7.82,76.3,0.63
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,-20.71,53.052776,0.528361,0.566667,-3.73,530.0,1.40
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,-7.38,50.705530,0.538721,0.400000,-9.77,NaN,0.28
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,-8.52,53.158862,0.496782,0.466667,-10.08,47.9,2.07


In [7]:
###Read in data from two separate experiments
teff_df1 = pd.read_excel('../Data/562_associated_data/msb188719-sup-0002-datasetev1.xlsx',\
                       sheet_name='LacZ')
teff_df2 = pd.read_excel('../Data/562_associated_data/msb188719-sup-0002-datasetev1.xlsx',\
                       sheet_name='PK-LacZ')

###And average them to calculate trans_eff_2019
teff_df1.columns = ['gene_id', 'teff1', 'teff2']
teff_df2.columns = ['gene_id', 'teff1', 'teff2']
teff_df = teff_df1.merge(teff_df2, on='gene_id')
teff_df['trans_eff_2019'] = np.mean(teff_df[['teff1_x', 'teff1_y']], axis=1)

###Append to full dataframe
teff_df = teff_df.drop(['teff1_x', 'teff1_y', 'teff2_x', 'teff2_y'], axis=1)
combined_df = combined_df.merge(teff_df, on='gene_id', how='left')
print(combined_df.shape)
combined_df.head()

(4018, 18)


,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,sec_struct_bound,iCUB,GC_cds,GC_upstream,Start_accessibility,prot_abundance,trans_eff_2014,trans_eff_2019
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,-12.81,55.949262,0.530654,0.433333,-8.50,317.0,0.98,NaN
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,-14.05,56.062386,0.562701,0.500000,-7.82,76.3,0.63,NaN
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,-20.71,53.052776,0.528361,0.566667,-3.73,530.0,1.40,NaN
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,-7.38,50.705530,0.538721,0.400000,-9.77,NaN,0.28,NaN
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,-8.52,53.158862,0.496782,0.466667,-10.08,47.9,2.07,0.410931


# Add in knowledge from two essential gene sets

Using two separate datasets for robustness. First:

https://doi.org/10.1128/mBio.02096-17

And then:
https://doi.org/10.1038/s41586-018-0124-0

In [8]:
essential_df = pd.read_csv('../Data/562_associated_data/ecoli_essential.txt', sep='\t')
essential_df.rename(columns={'Gene':'gene_id'}, inplace=True)
print('All:', essential_df.shape)
print('Essential:', essential_df[essential_df['Essential'] == True].shape)
print('Non-essential:', essential_df[essential_df['Non-essential'] == True].shape)
print('Unclear:', essential_df[essential_df['Unclear'] == True].shape)

###Re-factoring this dataframe a bit
essential_df['essentiality_ds1'] = ''
indices = essential_df[essential_df['Essential']==True].index
essential_df.loc[indices, 'essentiality_ds1'] = 'Essential'
indices = essential_df[essential_df['Non-essential']==True].index
essential_df.loc[indices, 'essentiality_ds1'] = 'Non-essential'
indices = essential_df[essential_df['Unclear']==True].index
essential_df.loc[indices, 'essentiality_ds1'] = 'Unclear'
print(essential_df['essentiality_ds1'].value_counts())

###Drop the unnecessary bits
essential_df = essential_df.drop(set(essential_df.columns)-set(['gene_id', 'essentiality_ds1']), axis=1)

###And combine
combined_df = combined_df.merge(essential_df, on='gene_id', how='left')
print(combined_df.shape)
combined_df.head()

All: (4313, 6)
Essential: (358, 6)
Non-essential: (3793, 6)
Unclear: (162, 6)
Non-essential    3793
Essential         358
Unclear           162
Name: essentiality_ds1, dtype: int64
(4018, 19)


,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,sec_struct_bound,iCUB,GC_cds,GC_upstream,Start_accessibility,prot_abundance,trans_eff_2014,trans_eff_2019,essentiality_ds1
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,-12.81,55.949262,0.530654,0.433333,-8.50,317.0,0.98,NaN,Non-essential
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,-14.05,56.062386,0.562701,0.500000,-7.82,76.3,0.63,NaN,Non-essential
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,-20.71,53.052776,0.528361,0.566667,-3.73,530.0,1.40,NaN,Non-essential
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,-7.38,50.705530,0.538721,0.400000,-9.77,NaN,0.28,NaN,Non-essential
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,-8.52,53.158862,0.496782,0.466667,-10.08,47.9,2.07,0.410931,Non-essential


In [9]:
essential_df = pd.read_csv('../Data/562_associated_data/ecoli_essential_Price.txt', sep='\t', skiprows=13)
print(essential_df.shape)

###Drop the unnecessary bits
essential_df['essentiality_ds2'] = 'Essential'
essential_df = essential_df.drop(set(essential_df.columns) - set(['sysName', 'essentiality_ds2']), axis=1)
essential_df.columns = ['locus_tag', 'essentiality_ds2']

###And merge
combined_df = combined_df.merge(essential_df, on='locus_tag', how='left')
print(combined_df.shape)
indices = combined_df[combined_df['essentiality_ds2']!='Essential'].index
combined_df.loc[indices, 'essentiality_ds2'] = 'Non-essential'
combined_df.head()

(324, 19)
(4018, 20)


,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,sec_struct_bound,iCUB,GC_cds,GC_upstream,Start_accessibility,prot_abundance,trans_eff_2014,trans_eff_2019,essentiality_ds1,essentiality_ds2
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,-12.81,55.949262,0.530654,0.433333,-8.50,317.0,0.98,NaN,Non-essential,Non-essential
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,-14.05,56.062386,0.562701,0.500000,-7.82,76.3,0.63,NaN,Non-essential,Non-essential
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,-20.71,53.052776,0.528361,0.566667,-3.73,530.0,1.40,NaN,Non-essential,Non-essential
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,-7.38,50.705530,0.538721,0.400000,-9.77,NaN,0.28,NaN,Non-essential,Non-essential
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,-8.52,53.158862,0.496782,0.466667,-10.08,47.9,2.07,0.410931,Non-essential,Non-essential


**Finally, add a consensus column of gene essentiality**

In [10]:
combined_df['essentiality'] = 'Unclear'
indices = combined_df[(combined_df['essentiality_ds1']=='Essential')&
                      (combined_df['essentiality_ds2']=='Essential')].index
combined_df.at[indices, 'essentiality'] = 'Essential'
indices = combined_df[(combined_df['essentiality_ds1']!='Essential')&
                      (combined_df['essentiality_ds2']!='Essential')].index
combined_df.at[indices, 'essentiality'] = 'Non-essential'

print(combined_df['essentiality'].value_counts())
combined_df.head()

Non-essential    3632
Essential         270
Unclear           116
Name: essentiality, dtype: int64


,Genome_source,Start,Stop,Strand,CDS_seq,Upstream_seq,locus_tag,gene_id,aSD_binding,sec_struct,...,iCUB,GC_cds,GC_upstream,Start_accessibility,prot_abundance,trans_eff_2014,trans_eff_2019,essentiality_ds1,essentiality_ds2,essentiality
0,NC_000913.3,336,2799,+,ATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAAC...,TTTTCGACCAAAGGTAACGAGGTAACAACC,b0002,thrA,-5.42,-21.31,...,55.949262,0.530654,0.433333,-8.50,317.0,0.98,NaN,Non-essential,Non-essential,Non-essential
1,NC_000913.3,2800,3733,+,ATGGTTAAAGTTTATGCCCCGGCTTCCAGTGCCAATATGAGCGTCG...,GTACCCTCTCATGGAAGTTAGGAGTCTGAC,b0003,thrB,-6.51,-21.87,...,56.062386,0.562701,0.500000,-7.82,76.3,0.63,NaN,Non-essential,Non-essential,Non-essential
2,NC_000913.3,3733,5020,+,ATGAAACTCTACAATCTGAAAGATCACAACGAGCAGGTCAGCTTTG...,ACGGCGGGCGCACGAGTACTGGAAAACTAA,b0004,thrC,-3.40,-24.44,...,53.052776,0.528361,0.566667,-3.73,530.0,1.40,NaN,Non-essential,Non-essential,Non-essential
3,NC_000913.3,5233,5530,+,GTGAAAAAGATGCAATCTATCGTACTCGCACTTTCCCTGGTTCTGG...,CATAACGGGCAATGATAAAAGGAGTAACCT,b0005,yaaX,-6.51,-17.15,...,50.705530,0.538721,0.400000,-9.77,NaN,0.28,NaN,Non-essential,Non-essential,Non-essential
4,NC_000913.3,5682,6459,-,ATGCTGATTCTTATTTCACCTGCGAAAACGCTTGATTACCAAAGCC...,GTCGGCATAAATTTCCTGCAAGGACTGGAT,b0006,yaaA,-4.00,-18.60,...,53.158862,0.496782,0.466667,-10.08,47.9,2.07,0.410931,Non-essential,Non-essential,Non-essential


# Save file

In [11]:
combined_df.to_csv('../Data/562_associated_data/ecoli.combined.tsv', sep='\t')